In [72]:
import numpy as np
from numpy import array
import pandas as pd
from pandas import DataFrame

In [91]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)-1):#遍历4511次
		# find the end of this pattern
		end_ix = i + n_steps_in #0+45, 1+45, 2+45...
		out_end_ix = end_ix + n_steps_out-1 #0+45+1-1, 1+45+1-1, 2+45+1-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[out_end_ix:out_end_ix+1, -1]
		seq_x = seq_x.flatten()
		seq_y = seq_y.flatten()
		# print(seq_y.shape)
		extra = sequences[end_ix:out_end_ix+1, 2:]
		# print(extra.shape)
		# extra[:,0:1] = 0
		# extra[:,22:25] = 0
		extra = extra.flatten()
		seq_x = np.concatenate([seq_x,extra], axis=0)
		# print(seq_x.shape)
        #seq_x = [0:0+45,:-1],[1:1+45,:-1],...
        #seq_y = [0+45:0+45+1-1+1,-1],[1+45:1+45+1-1+1,-1],...
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [92]:
# df = pd.read_excel('pingwangtest.xlsx',index_col=0)
df = pd.read_excel('/home/lzy/lab/Project/TaihuData/太湖处理.xlsx')
df.columns

Index(['TM', 'Z', 'Q(63202911)', 'Q(63205311)', 'DYP(42006400)',
       'DYP(42006000)', 'DYP(42005900)'],
      dtype='object')

In [93]:
in_cols = ['Z', 'Q(63202911)', 'Q(63205311)', 'DYP(42006400)',
       'DYP(42006000)', 'DYP(42005900)']
out_cols = ['Z']
df.shape

(8994, 7)

In [94]:
# choose a number of time steps
n_steps_in, n_steps_out = 10, 1

In [95]:
j=0
dataset_low = np.empty((df[out_cols[j]].values.shape[0],0))
print(dataset_low.shape)
for i in range(len(in_cols)):
    dataset_low = np.append(dataset_low, df[in_cols[i]].values.reshape(df[in_cols[i]].values.shape[0],1), axis=1)
dataset_low = np.append(dataset_low, df[out_cols[j]].values.reshape(df[out_cols[j]].values.shape[0],1), axis=1)
print(dataset_low.shape)
dataset_low = pd.DataFrame(dataset_low)
dataset_low = dataset_low.values

(8994, 0)
(8994, 7)


### 生成X与y，并与全部的TM组成一个DataFrame

In [96]:
X, y = split_sequences(dataset_low, n_steps_in, n_steps_out)
print(X.shape)
print(y.shape)
int(len(X[1]))
y = pd.DataFrame(y).rename(columns={0:int(len(X[1]))})

(8984, 65)
(8984, 1)


In [97]:
X[0]

array([2.91, 0.  , 0.  , 0.  , 0.  , 0.  , 2.91, 0.  , 0.  , 0.  , 0.  ,
       0.  , 2.91, 0.  , 0.  , 0.  , 0.  , 0.  , 2.9 , 0.  , 0.  , 0.  ,
       0.  , 0.  , 2.9 , 0.  , 0.  , 0.  , 0.  , 0.  , 2.9 , 0.  , 0.  ,
       0.  , 0.  , 0.  , 2.89, 0.  , 0.  , 0.  , 0.  , 0.  , 2.88, 0.  ,
       0.  , 0.  , 0.  , 0.  , 2.88, 0.  , 0.  , 0.  , 0.  , 0.  , 2.86,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.8 , 0.3 , 0.13, 2.86])

In [98]:
TM = df['TM'].iloc[n_steps_in+n_steps_out-1:].reset_index(drop=True)

In [99]:
TM

0      1999-01-11 08:00:00
1      1999-01-12 08:00:00
2      1999-01-13 08:00:00
3      1999-01-14 08:00:00
4      1999-01-15 08:00:00
               ...        
8979   2023-08-12 08:00:00
8980   2023-08-13 08:00:00
8981   2023-08-14 08:00:00
8982   2023-08-15 08:00:00
8983   2023-08-16 08:00:00
Name: TM, Length: 8984, dtype: datetime64[ns]

In [100]:
Xy = pd.concat([TM,pd.DataFrame(X),pd.DataFrame(y)],axis=1)
Xy.shape

(8984, 67)

In [101]:
start = Xy.iloc[:,0].iloc[0]
end = Xy.iloc[:,0].iloc[-1]
print(end)

t_index = pd.date_range(start=start, end=end, freq='D')
full_time = pd.DataFrame(t_index, columns=['TM'])

2023-08-16 08:00:00


### 抽出5-9月份的数据并进一步合并

In [102]:
five = full_time['TM'][full_time['TM'].dt.month == 5]
six = full_time['TM'][full_time['TM'].dt.month == 6]
seven = full_time['TM'][full_time['TM'].dt.month == 7]
eight = full_time['TM'][full_time['TM'].dt.month == 8]
nine = full_time['TM'][full_time['TM'].dt.month == 9]
flow_time = pd.concat([five,six,seven,eight,nine]).reset_index(drop=True)

In [103]:
flow_time = pd.DataFrame(flow_time).sort_values('TM')
flow_time

,TM
0,1999-05-01 08:00:00
1,1999-05-02 08:00:00
2,1999-05-03 08:00:00
3,1999-05-04 08:00:00
4,1999-05-05 08:00:00
...,...
3055,2023-08-12 08:00:00
3056,2023-08-13 08:00:00
3057,2023-08-14 08:00:00
3058,2023-08-15 08:00:00


In [104]:
flow_time.to_csv('/home/lzy/lab/Project/TaihuData/flow_time.csv')

In [105]:
complete = Xy.merge(flow_time,how='right',sort=True)
complete.shape

(3780, 67)

In [106]:
complete.to_csv('/home/lzy/lab/Project/TaihuData/TaihuDaily+1.csv')

In [307]:
i=1
ll=['a','b','c','e','f']
for it in ll:
 i_str=str(i)
 filename=i_str+'.txt'
 f=open( filename,'w')

 f.write("something")
 f.close()
 i=i+1

### 比对

In [308]:
x_ori = X[:,22]
x_com = df.iloc[:,5281]
set1 = set(x_ori)
set2 = set(x_com)
print(set1^set2)

IndexError: single positional indexer is out-of-bounds

In [ ]:
print(X.shape)
print(y.shape)

(93854, 6744)
(93854, 1)


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
pd.DataFrame(X)

0     1     2     3     4     5     6     7     8     9     10    11    \
0      2.76   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1      2.76   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2      2.76   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3      2.74   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4      2.73   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
93849  3.32   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93850  3.31   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93851  3.28   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93852  3.31   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93853  3.25   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

       12    13    14    15    16    17    18    19    20    21    22    23    \
0       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  2.98  2.76   
1       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  2.98  2.76   
2       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  2.98  2.75   
3       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  2.99  2.75   
4       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  2.99  2.74   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
93849   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  3.37  3.33   
93850   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  3.36  3.31   
93851   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  3.38  3.28   
93852   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  3.39  3.28   
93853   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  3.36  3.28   

       24    25    26    27    28    29    30    31    32    33    34    35    \
0      46.3  2.05  2.76   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1      49.5  1.97  2.76   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2      50.0  1.89  2.74   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3      48.1  1.82  2.73   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4      46.0  1.80  2.73   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
93849   0.0  2.37  3.31   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93850   0.0  2.34  3.28   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93851   0.0  2.51  3.31   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93852   0.0  2.94  3.25   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93853  41.2  3.16  3.26   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

       36    37    38    39    40    41    42    43    44    45    46    47    \
0       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
93849   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93850   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93851   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93852   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
93853   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

       48    49    50    51    52    53    54    55    56    57    58    59    \
0      2.98  2.76  49.5  1.97  2.7

In [ ]:
# print(X[0].shape) #(240, 25)
print(X.shape[1]) #(240, 25)

6744


In [ ]:
# X_col_name = []
# for i in range(0, 6240, 624):
#     name = X[i]
#     X[i] = name + 't-10'
#     X_col_name.append(['Z', 'DRP(63001000)', 'DRP(63203000)','DRP(63204401)',
#         'DRP(63204450)','DRP(63204600)','DRP(63204650)','DRP(63204800)','DRP(63204850)','DRP(63205350)','DRP(63300800)',
#         'DRP(63301150)','DRP(63301600)','DRP(63303600)','DRP(63304800)','DRP(63305200)','DRP(63305800)','DRP(63309200)','DRP(63309600)',
#         'DRP(63310600)','DRP(63403200)','DRP(63403500)','UPZ(太浦闸)','DWZ(太浦闸)','TGTQ(太浦闸)','TDZ(米市渡潮位)'])



In [ ]:
X = pd.DataFrame(X)
y = pd.DataFrame(y)
print(type(X))
X.to_csv('X_value.csv')
y.to_csv('y_value.csv')

<class 'pandas.core.frame.DataFrame'>


In [ ]:
np.savez('pinwang_Xy.npz',x=X,y=y)